<a href="https://colab.research.google.com/github/rraghavkaushik/Image-to-Image-Translation/blob/main/Image_to_Image_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy
import matplotlib.pyplot as plt
from keras.initializers import RandomNormal
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D, LeakyReLU, Activation, Concatenate, Dropout, BatchNormalization, Conv2DTranspose

In [10]:
def discriminator(img_size):
  wt = RandomNormal(stddev = 0.02)
  src_img = Input(shape = img_size)
  target_img = Input(shape = img_size)
  ''' concatenating channel wise'''
  conc_img = Concatenate()([src_img, target_img])

  ''' Discriminator architechture described in the paper :
    C64-C128-C256-C512
    After the last layer, a convolution is applied to map to
    a 1-dimensional output, followed by a Sigmoid function.
    As an exception to the above notation, BatchNorm is not
    applied to the first C64 layer. All ReLUs are leaky, with
    slope 0.2.'''


  l = Conv2D(64, (4, 4), strides = (2, 2), padding = 'same', kernel_initializer = wt)(conc_img)
  l = LeakyReLU(alpha = 0.2)(l)

  l = Conv2D(128, (4, 4), strides = (2, 2), padding = 'same', kernel_initializer = wt)(conc_img)
  l = BatchNormalization()(l)
  l = LeakyReLU(alpha = 0.2)(l)

  l = Conv2D(256, (4, 4), strides = (2, 2), padding = 'same', kernel_initializer = wt)(conc_img)
  l = BatchNormalization()(l)
  l = LeakyReLU(alpha = 0.2)(l)

  l = Conv2D(512, (4, 4), padding = 'same', kernel_initializer = wt)(l)
  l = BatchNormalization()(l)
  l = LeakyReLU(alpha = 0.2)(l)

  l = Conv2D(1, (4, 4), padding = 'same', kernel_initializer = wt)(l)
  output = Activation('sigmoid')(l)

  model = Model(inputs = [src_img, target_img], outputs = output)
  model.summary()
  model.compile(loss = 'binary_crossentropy', optimizer = Adam(lr = 0.0002, beta_1 = 0.5), metrics = ['accuracy'])
  return model
